In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
import numpy as np
import time

# ------------------ LucasBN manual eigenvalue functions ------------------
def get_dimensions(matrix):
    return [len(matrix), len(matrix[0])]

def list_multiply(list1, list2):
    # multiply two polynomial coefficient lists
    result = [0 for _ in range(len(list1) + len(list2) - 1)]
    for i in range(len(list1)):
        for j in range(len(list2)):
            result[i + j] += list1[i] * list2[j]
    return result

def list_add(list1, list2, sub=1):
    # element‑wise addition (or subtraction if sub=-1)
    return [i + (sub * j) for i, j in zip(list1, list2)]

def identity_matrix(dimensions):
    matrix = [[0 for _ in range(dimensions[1])] for _ in range(dimensions[0])]
    for i in range(dimensions[0]):
        matrix[i][i] = 1
    return matrix

def determinant_equation(matrix, excluded=[1, 0]):
    """
    Recursively build the characteristic polynomial coefficients
    in the form [c0, c1, ..., cn] representing c0 + c1*x + ... + cn*x^n.
    """
    dimensions = get_dimensions(matrix)
    if dimensions == [2, 2]:
        tmp = list_add(list_multiply(matrix[0][0], matrix[1][1]),
                       list_multiply(matrix[0][1], matrix[1][0]),
                       sub=-1)
        return list_multiply(tmp, excluded)
    else:
        new_matrices = []
        excl_vals = []
        exclude_row = 0
        for exclude_column in range(dimensions[1]):
            tmp = []
            excl_vals.append(matrix[exclude_row][exclude_column])
            for row in range(1, dimensions[0]):
                tmp_row = []
                for column in range(dimensions[1]):
                    if (row != exclude_row) and (column != exclude_column):
                        tmp_row.append(matrix[row][column])
                tmp.append(tmp_row)
            new_matrices.append(tmp)

        det_eqs = [determinant_equation(new_matrices[j], excl_vals[j])
                   for j in range(len(new_matrices))]
        # sum polynomials coefficient‑wise (padding shorter lists)
        max_len = max(len(p) for p in det_eqs)
        for p in det_eqs:
            p.extend([0]*(max_len-len(p)))
        dt_equation = [sum(coeffs) for coeffs in zip(*det_eqs)]
        return dt_equation

def characteristic_equation(matrix):
    return [[[a, -b] for a, b in zip(i, j)]
            for i, j in zip(matrix, identity_matrix(get_dimensions(matrix)))]

def find_eigenvalues_manual(matrix):
    coeffs = determinant_equation(characteristic_equation(matrix))
    roots = np.roots(coeffs[::-1])
    return roots

In [4]:
# ------------------ Matrix & timing ------------------
A = np.array([[6, 1, -1],
              [0, 7, 0],
              [3, -1, 2]], dtype=float)

# Manual eigenvalues timing
start_manual = time.perf_counter()
eig_manual = find_eigenvalues_manual(A.tolist())
manual_time = time.perf_counter() - start_manual

# NumPy's eig timing
start_np = time.perf_counter()
eig_np, _ = np.linalg.eig(A)
np_time = time.perf_counter() - start_np

eig_manual, eig_np, manual_time, np_time

(array([7., 5., 3.]),
 array([5., 3., 7.]),
 0.0004943079999293332,
 0.0001488759999119793)